# Job match application

Steps layout:

1. Turn the csv database into a python array with each job being a dictionary with the keys being the column values
2. Get a resume - convert from pdf python readable text using the pypdf module (to implement later)
   Next steps will be dependent on the type of feature

Add_job:

1. Use python csv module and just write another row of the job the user provided

- Do this on actual app - no need to do on notebook since easy

Search_job:

1. Have GPT list the job ids with the closest similarities to the resume text inputted
2. OTHER OPTION: Make the data into a vector database and get the ones with the closest cosine similarity

Scoring_job:
Get education_score, experience_score, skill_score

1. Feed resume into GPT and prompt engineer to give these scores back


# Unpack the csv file and print it out in a more python understadable format


In [9]:
import csv

# Put all the jobs into an array
jobs = {}
with open("./assets/data/raw_job_data.csv", "r", encoding="UTF-8") as file:
    reader = csv.DictReader(file)
    for row in reader:
        # Remove the unecessary job data - only need ID, title, description
        jobs[row["Job ID"]] = {"Title": row["Title"],
                               "Description": row["Description"]}

print(jobs)
print(len(jobs))

{'610cdc6f-4dcc-4be8-a629-be16d774d74f': {'Title': 'Project Coordinator (Digital Platform Project)', 'Description': 'What we want? \n· Coordinate project management activities, resources and information. \n· Meeting with different teams. \n· Monitor project progress and report project status. \n· Create and maintain project documentation, plan and reports. \n· Generate MOM and daily performance report. \n· Provide advice, assistance and follow-up on company policies, procedures and documentation. \n· Work on Site.\n\n\nWho are we looking for? \n·At least 1 - 2 years of experience supporting IT projects and experience in project management will be advantage. \n· Excellent communication, interpersonal and negotiation skills. \n· Excellent planning and organization ability. \n· Excellent self-learning, motivating and problem solving. \n· Team working and able to work under pressure with very good negotiation. \n· Excellent command of English.'}, 'c4e20c45-d9b7-4ce7-9f70-3eb1816a0c03': {'T

# Convert a PDF resume into text so that we can use it in our program

- If the PDF has readable text use the pymupdf module
- If the PDF cannot be read - conv each page of the PDF to an image and use OCR w/ easyOCR module to get the text


In [10]:
import pymupdf

text = ""
with pymupdf.open("./assets/resumes/functionalSample.pdf") as doc:
    for page in doc:
        text += page.get_text()  # type: ignore

print(text)

resumeText = text

Functional Resume Sample 
 
John W. Smith  
2002 Front Range Way Fort Collins, CO 80525  
jwsmith@colostate.edu 
 
Career Summary 
 
Four years experience in early childhood development with a diverse background in the care of 
special needs children and adults.  
  
Adult Care Experience 
 
• Determined work placement for 150 special needs adult clients.  
• Maintained client databases and records.  
• Coordinated client contact with local health care professionals on a monthly basis.     
• Managed 25 volunteer workers.     
 
Childcare Experience 
 
• Coordinated service assignments for 20 part-time counselors and 100 client families. 
• Oversaw daily activity and outing planning for 100 clients.  
• Assisted families of special needs clients with researching financial assistance and 
healthcare. 
• Assisted teachers with managing daily classroom activities.    
• Oversaw daily and special student activities.     
 
Employment History  
 
1999-2002 
Counseling Supervisor, The Wesley

# Search_Jobs

Let gpt find the top 5 closest matching jobs for the resume in text form


Variation 1 - doesn't work well

- Might need to go with a more systematic approach - a more logical algorithm that has specific instructions


In [20]:
from openai import AzureOpenAI
from dotenv import dotenv_values

secrets = dotenv_values("../.env")

client = AzureOpenAI(azure_endpoint=secrets["AZURE_OPENAI_ENDPOINT"],  # type: ignore
                     api_version=secrets["AZURE_OPENAI_API_VERSION"],
                     api_key=secrets["AZURE_OPENAI_API_KEY"],
                     azure_deployment="gpt4o")


def searchJobs(userResume: str, resultsAmount: int):

    instructionPrompt = [{"role": "system", "content": "You are a company's HR assistant. Your duty is to help search for the right job positions for a candidate by comparing all the job positions available with the candidate's job resume and returning the jobs which are the most fitting for the candidate's resume."},

                         {"role": "system", "content": f"Here are all the available job positions and their descriptions: \n{
                             jobs}"},

                         {"role": "system", "content": f"Here is the candidate's resume: \n{
                             userResume}"},

                         {"role": "system", "content": """

						Use these judgement factor as a reference to go off of for your analysis of which job fits the user's resume the most(they are not listed in order of importance): 

						- Job relevance: How closely does the candidate's work experience align with the job requirements?

						- Skill alignment: Does the candidate possess the necessary skills and competencies? The more skillful a candidate in that job's respective field, the more compatible that candidate is for jobs in that field. If a candidate does not contain the required skills for a job, they are not compatible for the job and should be ruled out. 

						- Education and qualifications: Does the candidate meet the minimum education and qualification standards? Candidates that have education in a field will be more compatible with jobs in that same respective field. 

						- Experience level: Does the candidate's work experience match the required experience level? If the candidate has a lot of experience in a particular field, they will be highly compatible with jobs in the same respective field and should be recommended a high status job in that field.

						- Keyword matching: How well do the keywords in the resume match the keywords in the job descriptions? A high compatibility individual will have many keywords in their resume which match the job description. This field should be weighed pretty highly. If a resume contains a lot of similar words, the candidate would be very compatible with the job. 

						- Heuristic analysis: Is this job the best use of the user's abilities in this company or are there other jobs which fit the user's skills, abilities, and experiences better? Would a candidate of this experience or skill level be hard to find?(If they are hard to find, then they should be recommended more high level job positions)"""},

                         {"role": "system", "content": """Here are the steps in which you should use the resources provided up to determine whether or not the user's resume is fitting for a particular job:

	   					1. Start iterating each job positions. 

	   					2. While iterating through each job, rank the provided candidate's resume with each judgement factor in a scale of compatibility out of 10 from the compatibility of their resume with that specific job description based on that one judgement factor point. Store those ranking values in a python list nested inside a python dictionary named rankingValues. The keys should be the title of the job position while the value should be the list with the ranked values. 

	   					3. Once you've iterated through all the jobs, take the scores in each list inside the dictionary and tally them up per each list, making a combined compatibility value per job. Store the combined compatibility values per job inside a python list named combineValues. They should be stored in tuple pairs for each index of the list with the first tuple index value being the job position name while the other being the combined compatibiilty value score of that job position that was calculated in the previous steps. 

	   					4. Take the combineValues list and sort it from the highest compatibility value to the lowest. 

	   					5. When the user requests n number of top search results, take the job positions at the first n values of the sorted combineValues list and use those as the top matching job positions. Return the top matching job positions for the user in the instructed format.

						"""},

                         {"role": "system",
                          "content": "Your return message to the user should follow EXACTLY in format that will be described. It will be in the form of Python dictionaries nested in the Python list datatype in which you only need to return the Job ID and the Job Title inside each of the dictionaries. Simply return the job positions which fit best with the resume based on the prompts that were previously given to you. Do it in order - starting with the jobs that fit the best with the resume. Your return should NOT be in Python code. It should strictly be in the text format. Here is an example for the return format. [{'Job ID': Job ID, 'Job Title': Job Title}, {'Job ID': Job ID, 'Job Title': Job Title}]"},
                         {"role": "system", "content": "Under no circumstances should you recommend a job to a candidate which they have no experience or education in. You must prioritize the jobs which an individual has experience in. Individuals with longer duration of time they've worked before(such as 2+ years) should be recommended higher tier jobs(and not internships). "}]

    completion = client.chat.completions.create(model="gpt-4o", messages=instructionPrompt + [
        # type: ignore
        {"role": "system", "content": f"Please return the top {resultsAmount} results for the search. "}])

    return completion.choices[0].message.content


print(searchJobs(resumeText, 5))

[{'Job ID': 'e497b8c8-a1bd-4a4c-8f55-368bfb16628d', 'Job Title': 'IT Support Officer'}, {'Job ID': 'ef87e4ae-1d2c-4a75-9577-80954d1dd98a', 'Job Title': 'System Operator (Onsite)'}, {'Job ID': '610cdc6f-4dcc-4be8-a629-be16d774d74f', 'Job Title': 'Project Coordinator (Digital Platform Project)'}, {'Job ID': '629dc125-a68e-42cf-bad9-c99229e7f155', 'Job Title': 'Information Security Implementation Engineer (Network Security)'}, {'Job ID': 'e6de0fa5-fb07-4fbd-96e2-91b06991c1d7', 'Job Title': 'System Engineer'}]


Variation 2

Shorter prompts but still doesn't work that well


In [19]:
def searchJobs(userResume: str, resultsAmount: int):
    instructionPrompt = [{"role": "system", "content": "You are a company's HR assistant. Your duty is to help search for the right job positions for a candidate by comparing all the job positions available with the candidate's job resume and returning the jobs which are the most fitting for the candidate's resume."},
                         {"role": "system", "content": f"Here are all the available job positions and their descriptions: \n{
                             jobs}"},
                         {"role": "system", "content": f"Here is the candidate's resume: \n{
                             userResume}"},
                         {"role": "system", "content": "Please have the reasoning for your recommendation for the jobs that you chose for the candidate written. However, you do not need to return it with the return message. "},
                         {"role": "system",
                          "content": "Your return message to the user should follow EXACTLY in format that will be described. It will be in the form of Python dictionaries nested in the Python list datatype in which you only need to return the Job ID and the Job Title inside each of the dictionaries. Simply return the job positions which fit best with the resume based on the prompts that were previously given to you. Do it in order - starting with the jobs that fit the best with the resume. Your return should NOT be in Python code. It should strictly be in the text format. Here is an example for the return format. [{'Job ID': Job ID, 'Job Title': Job Title}, {'Job ID': Job ID, 'Job Title': Job Title}]"},
                         {"role": "system", "content": "Under no circumstances should you recommend a job to a candidate which they have no experience or education in. You must prioritize the jobs which an individual has experience in. Individuals with longer duration of time they've worked before(such as 2+ years) should be recommended higher tier jobs(and not internships). "}]

    completion = client.chat.completions.create(model="gpt-4o", messages=instructionPrompt + [
        # type: ignore
        {"role": "system", "content": f"Please return the top {resultsAmount} results for the search. "}])
    return completion.choices[0].message.content


print(searchJobs(resumeText, 5))

[{'Job ID': '3d5dcbbe-1c93-4d42-95fa-b664c6288a7c', 'Job Title': 'Business Development Internship'}, {'Job ID': '8d1912d0-b236-447a-954e-36f6722137fb', 'Job Title': 'Business Analyst/Business Consultant'}, {'Job ID': '1cdc21ea-7842-4078-9180-19481b39b9b2', 'Job Title': 'Software Tester (Automate test)'}, {'Job ID': '84705cdc-187b-4f03-a45d-4fe277a8fd41', 'Job Title': 'Technical Consultant (Presales) - Data Solutions'}, {'Job ID': '9f66cca6-23ac-4e44-9e7e-241f74e1c1b0', 'Job Title': 'Software Engineer (Software Implementer)'}]


Variation 3: Sort of works but still not as good as I'd like it to be


In [8]:
def searchJobs(userResume: str, resultsAmount: int):
    instructionPrompt = [{"role": "system", "content": "You are a company's HR assistant. You will read through a candidate's resume and return a python list in text format of the most important key words and knowledge from the resume in order to assist with choosing an appropriate position for the candidate at the company. Be sure to include key words about the client's previous work or education experience and the recommended caliber/authority level of a compatible job. Also include potential names for any suitable job positions or fields for the key words too. Try to include as many key words as possible(preferably at least 10). "},
                         {"role": "system", "content": f"Here is the candidate's resume: \n{
                             userResume}"},
                         {"role": "system", "content": "Your keywords should strictly be in this format as shown from an example: ['Engineer', 'Manager', 'Computer', 'Electrical Engineer', 'Project Leader']"}]

    resumeKeyWords = client.chat.completions.create(
        # type: ignore
        model="gpt-4o", messages=instructionPrompt).choices[0].message.content

    answer = client.chat.completions.create(model="gpt-4o", messages=[
        {"role": "system", "content": "You are a company's HR manager. You will receive some key words from a candidate's resume and a list of available job positions at the company. Your job is to return a list of the most compatible jobs based on the key words provided from the candidate's resume. "},
        {"role": "system",
         "content": f"Here are the available jobs in the company: \n{jobs}"},
        {"role": "system", "content": f"Here are the key words from the user's resume: {
            resumeKeyWords}"},
        {"role": "system", "content": "Make sure to the return the most compatible jobs that fall in line with the keywords listed. Prioritize recommending more high quality jobs first and jobs that relate the most to the key words. "},
        {"role": "system",
         "content": "Your return message to the user should follow EXACTLY in format that will be described. It will be in the form of Python dictionaries nested in the Python list datatype in which you only need to return the Job ID and the Job Title inside each of the dictionaries. Simply return the job positions which fit best with the resume based on the prompts that were previously given to you. Do it in order - starting with the jobs that fit the best with the resume. Your return should NOT be in Python code. It should strictly be in the text format. Here is an example for the return format. [{'Job ID': Job ID, 'Job Title': Job Title}, {'Job ID': Job ID, 'Job Title': Job Title}]"},
        {"role": "user", "content": f"Return the top {
            resultsAmount} results for matching jobs. "}
    ])

    return answer.choices[0].message.content


print(searchJobs(resumeText, 5))

[{'Job ID': '059d1ddc-61f7-4a16-95c8-79d007e6db3e', 'Job Title': 'Legal Documentation Assistant Intern'}, {'Job ID': '186ae342-3e59-484a-8db3-b0a28e1c554a', 'Job Title': 'IoT and Smart Solution Internship'}, {'Job ID': '3d5dcbbe-1c93-4d42-95fa-b664c6288a7c', 'Job Title': 'Business Development Internship'}, {'Job ID': '445f6122-4f93-437d-b554-15172f8bd98e', 'Job Title': 'Technical Consultant (Infra & BigData)'}, {'Job ID': '8d1912d0-b236-447a-954e-36f6722137fb', 'Job Title': 'Business Analyst/Business Consultant'}]


Most all-in-one GPT algorithm finder functions don't work well

- Algorithm to help minimize the size that GPT has to look through

1. Have GPT find the keywords of the most important key words of the resume
2. Have GPT split the jobs into categories
3. Have GPT find the closest job categories for the resume keywords
4. Find the closest job positions that are inside the job category for the resume


In [11]:
from openai import AzureOpenAI
from dotenv import dotenv_values

secrets = dotenv_values("./.env")

client = AzureOpenAI(azure_endpoint=secrets["AZURE_OPENAI_ENDPOINT"],  # type: ignore
                     api_version=secrets["AZURE_OPENAI_API_VERSION"],
                     api_key=secrets["AZURE_OPENAI_API_KEY"],
                     azure_deployment="gpt4o")

Sorting job into categories in order to optimize speed of job matching


In [13]:
import json


def sortJobIntoCategories(jobs, client) -> dict:
    # Generate the job categories to optimize search times
    instructionPrompt = [{"role": "system", "content": f"""You will be given a list of jobs - sort them into different job categories which you deem fit for the job. Do not sort them into more than 8 cateogories.
                                                                       Under no circumstances should you make a separate category purely for Internships. DO NOT MAKE A SEPARATE CATEGORY FOR INTERNSHIPS. Under no circumstances will you make a separate category for 'Miscellaneous' - you must put the jobs into a category.
                                                                       Here are the jobs that you should sort: {jobs}""" + "\nYour return format for the sorted jobs inside their categories should be python dictionaries nested inside the main python dictionary(but return it in pure text and only in the format shown - you do not need to show it as code in python or anything - just give the dictionary in it's pure form). Under no circumstances should you add any other text other than the answer you should give. It should be in the format exacty like this(make sure not to foget the ' character): {'Job Category': {'Job Title': 'Job ID', 'Job Title': 'Job ID'}, 'Job Category': {'Job Title': 'Job ID', 'Job Title': 'Job ID'} }"}]

    global sortedJobs
    sortedJobs = client.chat.completions.create(
        # type: ignore
        model="gpt-4-turbo", messages=instructionPrompt).choices[0].message.content
    # Convert the dictionary returned into an usable format(not string)
    exec(f"sortedJobs = {sortedJobs}", globals())

    # Save the sorted jobs
    with open("./assets/data/categorized_job_titles.jsonl", "w", encoding="UTF-8") as file:
        json.dump(sortedJobs, file, ensure_ascii=False)

    return sortedJobs


jobs_sorted = sortJobIntoCategories(jobs=jobs, client=client)


print(jobs_sorted.keys())

dict_keys(['IT Project Management', 'Software Development', 'Database Systems', 'System and Network Administration', 'Data and Analytics', 'Business and Financial Analysis', 'Creative and Marketing', 'Sales and Customer Support'])


Run this cell if you need to get the sorted jobs


In [9]:


print(jobs_sorted.keys())

dict_keys(['Project Management', 'Software Engineering', 'Database Administration', 'System Engineering', 'Business Development', 'Marketing & Communications', 'Consulting', 'Graphic Design', 'IT Support', 'Network Engineering', 'Accounting & Finance', 'Human Resources', 'Cyber Security', 'Operations', 'Deployment & Support', 'Development', 'Legal', 'Purchasing', 'Consulting and Strategy', 'Data Engineering'])


Final Variation

This variation works the best

- Learned that the most effect prompts for GPT are ones that are concise, yet short(don't micromanage the tasks). They should also just be inside one entire string - don't need to separate into multiple messages since it'll just cause more hallucinations. 

In [13]:
def searchJobs(userResume: str, resultsAmount: int, sortedJobs):
    # Get the resume, and find the appropriate job categories from the list of job categories
    instructionPrompt = [{"role": "system", "content": "You will be given job categories and a resume. Find the job categories which are most compatible for the resume given. Return ONLY the suitable job cateogories in this example format: ['Engineering', 'System Informations', 'Psychology']. Do not return any other text except that. " +
                          f"""Here is the resume: \n{userResume}Here are the job categories: \n{list(sortedJobs.keys())}"""}]
    global chosenCategories
    chosenCategories = client.chat.completions.create(
        # type: ignore
        model="gpt-4o", messages=instructionPrompt).choices[0].message.content
    # Set it into a python recognizable list while making sure it's also inside the sortedJobs list
    exec(f"chosenCategories = [category for category in {
         chosenCategories} if category in sortedJobs.keys()]", globals())
    # Return all the jobs in the requested categories
    # sortedJobs.values = all {job title: job ID} values
    # Make a dictionary of all the jobs that are part of the chosen categories(combined)
    chosenJobs = {}
    for category in chosenCategories:
        for jobTitle in sortedJobs[category]:
            jobID = sortedJobs[category][jobTitle]
            chosenJobs[jobID] = {"Title": jobTitle,
                                 "Description": jobs[jobID]["Description"]}

    instructionPrompt = [{"role": "system", "content": f"""Your job is to choose the most compatible jobs inside a provided list of jobs(and job descriptions for that job) based on a candidate's resume that is given.
                                                                                    Keep in mind that the 'Special assignment' descriptor inside the job descriptions indicate that it is a high level job. You should only recommend high level positions if the candidate is qualified and has a lot of experience.
                                                                                    Here is the candidate's resume: \n{userResume}
                                                                                    \nPlease return the top {resultsAmount} jobs which suit the candidate the best. \n"""+"Here is a dictionary of jobs inside the categories that suits the user's resume(Each set in the dictionary is in this format - Job ID: {'Title': Job title, 'Description': Job Description): \n" + f"{chosenJobs}\n" + "Your return message to the user should follow EXACTLY in format that will be described. It will be in the form of Python dictionaries nested in the Python list datatype in which you only need to return the Job ID and the Job Title inside each of the dictionaries. Your return should NOT be in Python code. It should strictly be in the text format. Here is an example for the return format. [{'Job ID': Job ID, 'Job Title': Job Title} , {'Job ID': Job ID, 'Job Title': Job Title}]"}]

    global selectedJobs
    selectedJobs = client.chat.completions.create(
        # type: ignore
        model="gpt-4o", messages=instructionPrompt).choices[0].message.content
    exec(f"selectedJobs = {selectedJobs}", globals())

    return {"status": 200, "message": "Matching jobs successfully found", "content": selectedJobs}


# Get the sorted jobs from the json categorized_job_titles.jsonl in assets first
print(searchJobs(resumeText, 5, jobs_sorted))

{'status': 200, 'message': 'Matching jobs successfully found'}


# Job Scoring algorithm

- Have GPT generate a grading scale for user reference along with for the AI to reference
- Parameters: Job to score, Resume of the user


In [22]:
def score_job(jobSet, userResume: str):
    instructionPrompt = [{"role": "system", "content": f"""You will be given a job and it's description along with a candidate's resume for the job. Your purpose is to score the candidate's compatibiility with the job and give out a score for the Education, Skills, and Experience categories.
                                            Here is a grading scale that you can go off of for the scoring(keep in mind this is only something to go off of. Scoring requirements should not be the same for every job. Use your own judgement capabilities to score with this grading scale as a simple reference. For example, applying for a software tester job won't have as much educational requirements as applying for a data analyst job):
                                            Grading Scale
                                            Education Score (0-10)
                                            10: Candidate possesses an advanced degree (e.g., PhD, MD) directly relevant to the position.
                                            5: Candidate holds a bachelor's degree in a related field.
                                            0: Candidate lacks any formal education or equivalent qualifications.
                                            Skill Score (0-10)
                                            10: Candidate possesses an exceptional level of proficiency in all required skills.
                                            5: Candidate has basic proficiency in required skills, but may need additional training or development.
                                            0: Candidate possesses few or no relevant skills.
                                            Experience Score (0-10)
                                            10: Candidate has extensive, directly relevant experience in a similar role, demonstrating exceptional performance and achievements.
                                            5: Candidate has some experience in related fields, but may require additional on-the-job training.
                                            0: Candidate has no identifiable work experience.
                                            Here is the job to analyze: {jobSet}\nHere is the candidate's resume: {userResume}"""
                          + "Your return format should STRICTLY be in the format that will be shown. There should be no other text or reply except in this format for the scoring: {'Education': 5, 'Skills': 10, 'Experience': 9}"}]

    global scoreDict
    scoreDict = client.chat.completions.create(
        # type: ignore
        model="gpt-4o", messages=instructionPrompt).choices[0].message.content
    exec(f"scoreDict = {scoreDict}", globals())
    # Http code 200 - content successfully retrieved
    return {"status": 200, "message": "Score evaluation successful", "content": scoreDict}


print(score_job(
    jobSet=jobs["0fcccb68-3c78-41c3-bdcb-04b2d77c4466"], userResume=resumeText))

{'status': 200, 'message': 'Score evaluation successful', 'content': {'Education': 0, 'Skills': 0, 'Experience': 0}}


# Add files function

- Add a job to the raw job data file
- Re-generate the job categories


In [50]:
import csv


def add_job(jobDatabasePath: str, job_id: str, job_title: str, job_description: str, openAIclient) -> dict:
    jobDatabase = {}
    # Get the jobs
    with open(jobDatabasePath, "r", encoding="UTF-8") as file:
        reader = csv.DictReader(file)
        for row in reader:
            # Remove the unecessary job data - only need ID, title, description
            jobDatabase[row["Job ID"]] = {
                "Title": row["Title"], "Description": row["Description"]}

    # check if the job already exists in the database(based off of job_id and job name)
    jobTitles = [value["Title"] for value in jobDatabase.values()]
    if (job_id in jobDatabase) or (job_title in jobTitles):
        # Http status code 409 - conflict since the job position already exists
        return {"status": 409, "message": "Conflict since the job position already exists"}

    # Add the new job into the database
    with open(jobDatabasePath, "a", encoding="UTF-8") as file:
        writer = csv.DictWriter(
            file, fieldnames=["Job ID", "Title", "Description"])
        writer.writerow({"Job ID": job_id, "Title": job_title,
                        "Description": job_description})

    # DO NOT run this for test cases
    # Re-optimize the database to incorporate new data(splits the database into groups)
    # More efficient way to do this would just be to add it into the correct place in the same json file, but this is just a demo so to save coding time, we will just re-generate the optimizer file
    sortJobIntoCategories(jobs=jobDatabase, client=openAIclient)

    # Return indicating a successful adding of the jobs
    # Http code 201 - created successfully
    return {"status": 409, "message": "New job added successfully"}


print(add_job("./assets/data/raw_job_data.csv",
      "test1234", "test", "whatever description", client))

{'status': 409, 'message': 'Conflict since the job position already exists'}


# API Testing - api code complete


Score jobs function

In [47]:
import requests

with open("./assets/resumes/resume_sample.pdf", "rb") as file:
    response = requests.post("http://127.0.0.1:8000/score_job/", files={
                             "resumePdfFile": file}, params={"job_id": "57babd41-e03e-4ab4-991c-d9fe59ac506f"})
    print(response.content)

b'{"status":200,"message":"Score evaluation with job(People Benefits and Well-Being Internship) and resume was successful","content":{"Education":5,"Skills":5,"Experience":4}}'


Search jobs function

In [49]:
# Test search jobs

with open("./assets/resumes/functionalSample.pdf", "rb") as file:
    response = requests.post(
        "http://127.0.0.1:8000/search_job/", files={"resumePdfFile": file})
    print(response.content)

b'{"status":200,"message":"Matching jobs successfully found","content":[{"Job ID":"0bfbb59a-b5b2-41c8-876e-e15ae3be1fbe","Job Title":"Innovation Consultant"},{"Job ID":"490f3b59-29b6-419f-a701-c349729bfe69","Job Title":"Project Engineer"},{"Job ID":"85aea70c-29ee-4a86-b313-2a75dfb69557","Job Title":"Assistance customer success manager (Streaming Platform and development)"},{"Job ID":"ecb5c77b-2152-456f-8027-a38fe9659ba7","Job Title":"Helpdesk Engineer"},{"Job ID":"20ff36c1-e57a-4d0d-8dde-5c2511d738a9","Job Title":"Customer Support Engineer"}]}'
